In [ ]:
import time
import csv
import pickle

import operator
item_to_cat = {}

with open("./product-categories.csv") as f:
    reader = csv.DictReader(f, delimiter=';')
    for data in reader:
        item_to_cat[data['itemId']] = data['categoryId']

In [ ]:
import time
import csv
import pickle

import operator

# Load the diginetica dataset
with open("./train-item-views.csv") as f:
    reader = csv.DictReader(f, delimiter=';')
    sess_clicks = {}
    # Only keep the 'starting' date for each session
    sess_date = {}
    ctr = 0
    curid = -1
    curdate = None
    for data in reader:
        sessid = data['sessionId']
        if curdate and not curid == sessid:
            date = time.mktime(time.strptime(curdate, '%Y-%m-%d'))
            sess_date[curid] = date
        curid = sessid
        item = data['itemId']
        curdate = data['eventdate']
        timeframe = int(data['timeframe'])
        if sessid in sess_clicks:
            sess_clicks[sessid] += [(timeframe, item)]
        else:
            sess_clicks[sessid] = [(timeframe, item)]
        ctr += 1
        if ctr % 100000 == 0:
            print ('Loaded', ctr)
    date = time.mktime(time.strptime(curdate, '%Y-%m-%d'))
    sess_date[curid] = date

for s in sess_clicks.keys():
    l = sorted(sess_clicks[s])
    sess_clicks[s] = [x[1] for x in l]

# Filter out length 1 sessions
to_be_deleted = []
for s in sess_clicks.keys():
    if len(sess_clicks[s]) == 1:
        to_be_deleted.append(s)
for s in to_be_deleted:
    del sess_clicks[s]
    del sess_date[s]

# Count number of times each item appears
iid_counts = {}
for s in sess_clicks:
    seq = sess_clicks[s]
    for iid in seq:
        if iid in iid_counts:
            iid_counts[iid] += 1
        else:
            iid_counts[iid] = 1

sorted_counts = sorted(iid_counts.items(), key=operator.itemgetter(1))

to_be_deleted = []
for s in sess_clicks.keys():
    curseq = sess_clicks[s]
    filseq = list(filter(lambda i: iid_counts[i] >= 5, curseq))
    if len(filseq) < 2:
        to_be_deleted.append(s)
    else:
        sess_clicks[s] = filseq

for s in to_be_deleted:
    del sess_clicks[s]
    del sess_date[s]

# Split out test set based on dates
dates = list(sess_date.items())
maxdate = dates[0][1]

for _, date in dates:
    if maxdate < date:
        maxdate = date

# 7 days for test
splitdate = maxdate - 86400 * 7
print('Split date', splitdate)
train_sess = filter(lambda x: x[1] < splitdate, dates)
test_sess = filter(lambda x: x[1] > splitdate, dates)

# Sort sessions by date
train_sess = sorted(train_sess, key=operator.itemgetter(1))
test_sess = sorted(test_sess, key=operator.itemgetter(1))



Loaded 100000
Loaded 200000
Loaded 300000
Loaded 400000
Loaded 500000
Loaded 600000
Loaded 700000
Loaded 800000
Loaded 900000
Loaded 1000000
Loaded 1100000
Loaded 1200000
Split date 1464134400.0


In [ ]:
# Choosing item count >=5 gives approximately the same number of items as reported in paper
item_dict = {}
item_ctr = 1
category_dict = {}
category_ctr = 1
train_seqs = []
train_cat = []
train_dates = []

id_to_cat_model = {}
# Convert training sessions to sequences and renumber items to start from 1
for s, date in train_sess:
    seq = sess_clicks[s]
    outseq = []
    cat = []
    for i in seq:
        if i in item_dict:
            outseq += [item_dict[i]]
            cat += [category_dict[item_to_cat[i]]]
        else:
            outseq += [item_ctr]
            item_dict[i] = item_ctr
            item_ctr += 1
            if item_to_cat[i] in category_dict:
                id_to_cat_model[item_dict[i]] = category_dict[item_to_cat[i]]
                cat += [category_dict[item_to_cat[i]]]
            else:
                cat += [category_ctr]
                category_dict[item_to_cat[i]] = category_ctr
                category_ctr += 1
                id_to_cat_model[item_dict[i]] = category_dict[item_to_cat[i]]
    if len(outseq) < 2:  # Doesn't occur
        continue
    train_seqs += [outseq]
    train_dates += [date]
    train_cat += [cat]

test_seqs = []
test_dates = []
test_cat = []
# Convert test sessions to sequences, ignoring items that do not appear in training set
for s, date in test_sess:
    seq = sess_clicks[s]
    outseq = []
    cat = []
    for i in seq:
        if i in item_dict:
            outseq += [item_dict[i]]
            cat += [category_dict[item_to_cat[i]]]
    if len(outseq) < 2:
        continue
    test_seqs += [outseq]
    test_dates += [date]
    test_cat += [cat]

print(item_ctr)
print(category_ctr)

43098
996


In [ ]:
def process_seqs(iseqs, idates):
    out_seqs = []
    out_dates = []
    labs = []
    for seq, date in zip(iseqs, idates):
        for i in range(1, len(seq)):
            tar = seq[-i]
            labs += [tar]
            out_seqs += [seq[:-i]]
            out_dates += [date]

    return out_seqs, out_dates, labs


tr_seqs_side, tr_dates_side, tr_labs_side = process_seqs(train_cat,train_dates)
te_seqs_side, te_dates_side, te_labs_side = process_seqs(test_cat,test_dates)

train_side = (tr_seqs_side, tr_labs_side)
test_side = (te_seqs_side, te_labs_side)

f1 = open('./digi_train_side.pkl', 'wb')
pickle.dump(train_side, f1)
f1.close()
f2 = open('./digi_test_side.pkl', 'wb')
pickle.dump(test_side, f2)
f2.close()

print('Done.') 

Done.


In [ ]:
tr_seqs, tr_dates, tr_labs = process_seqs(train_seqs,train_dates)
te_seqs, te_dates, te_labs = process_seqs(test_seqs,test_dates)

train = (tr_seqs, tr_labs)
test = (te_seqs, te_labs)

f1 = open('./digi_train.pkl', 'wb')
pickle.dump(train, f1)
f1.close()
f2 = open('./digi_test.pkl', 'wb')
pickle.dump(test, f2)
f2.close()

print('Done.') 

Done.


In [ ]:
l = []
for i in range(1, 43098):
    l.append(id_to_cat_model[i])

In [ ]:
import numpy as np
index_list = np.array(l)

In [ ]:
f2 = open('./digi_side.pkl', 'wb')
pickle.dump(index_list, f2)
f2.close()